In [15]:
!pip install sagemaker
!pip install boto3



In [3]:
import boto3

sts_client = boto3.client('sts')
# Call the get-caller-identity method to test the connection
response = sts_client.get_caller_identity()

In [142]:
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

def get_execution_role():
    try:
        print('starting execution_role...')
        role = sagemaker.get_execution_role()
        print('starting execution_role...OK')
    except ValueError:
        print('error execution_role...')
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker-execution-role')['Role']['Arn']
        print('error execution_role...OK')

    print(f"sagemaker role arn: {role}")
    return role

def getHuggingFaceModel(role):	
	# create Hugging Face Model Class
	huggingface_model = HuggingFaceModel(
	   model_data="s3://my-ml-models-123/model_t5_medical_chatbot_3.tar.gz",  # path to your trained sagemaker model
	   role=role, # iam role with permissions to create an Endpoint
	   transformers_version="4.37.0", # transformers version used
	   pytorch_version="2.1.0", # pytorch version used
	   py_version="py310", # python version of the DLC
       env={"HF_TASK": "summarization"}
	)
	return huggingface_model

def deploy(huggingface_model):
	predictor = huggingface_model.deploy(
	   initial_instance_count=1,
	   instance_type="ml.t2.medium"
	)
	return predictor

def predict(predictor, question):
    # example request: you always need to define "inputs"
    data = {
       "inputs": question,
        "parameters": {
            "max_length": 500,  # Increase max_length here
            "num_return_sequences": 1
        }
    }
    # request
    print(predictor.predict(data))

In [143]:
role = get_execution_role()

starting execution_role...


[02/04/25 11:48:22] WARNING  Couldn't call 'get_role' to get Role ARN from role name admin to get   ]8;id=898388;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=34622;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#5971\5971]8;;\
                             Role path.                                                                            

error execution_role...
error execution_role...OK
sagemaker role arn: arn:aws:iam::756553336060:role/sagemaker-execution-role


In [144]:
huggingface_model = getHuggingFaceModel(role)

In [145]:
predictor = deploy(huggingface_model)

[02/04/25 11:48:28] INFO     Creating model with name:                                              ]8;id=478801;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=684000;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\
                             huggingface-pytorch-inference-2025-02-04-16-48-28-406                                 

[02/04/25 11:48:29] INFO     Creating endpoint-config with name                                     ]8;id=476805;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=627217;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#5889\5889]8;;\
                             huggingface-pytorch-inference-2025-02-04-16-48-29-240                                 

                    INFO     Creating endpoint with name                                            ]8;id=533523;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=720576;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#4711\4711]8;;\
                             huggingface-pytorch-inference-2025-02-04-16-48-29-240                                 

------------------!

In [140]:
predict(predictor, 
		"What should I do to reduce my weight gained due to genetic hypothyroidism?")

[{'generated_text': 'Hi, dairy have gone through your diet and diet. You should go for diet and exercise. You should go for diet and exercise. You should go for diet and exercise. You should go for diet and exercise. You should take a course of antibiotics like a steroid like a steroid like a steroid like a steroid like a steroid like a steroid like a steroid like a steroid like a steroid like a steroid like a steroid.'}]


In [141]:
predictor.delete_endpoint()

[02/04/25 11:47:56] INFO     Deleting endpoint configuration with name:                             ]8;id=62234;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=492060;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#4865\4865]8;;\
                             huggingface-pytorch-inference-2025-02-04-15-26-53-559                                 

                    INFO     Deleting endpoint with name:                                           ]8;id=256263;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=599107;file:///mnt/c/Users/ericr/git/ai-dl-math/python-assistant/myenv/lib/python3.12/site-packages/sagemaker/session.py#4855\4855]8;;\
                             huggingface-pytorch-inference-2025-02-04-15-26-53-559                                 